#### Install binary dependencies and required python modules:
* CMake
* CFFI
* Cython
* PyBind11
* numpy

In [3]:
from sys import platform
import logging as log
import subprocess

log.basicConfig(format='[ %(levelname)s ] %(message)s', level=log.DEBUG)

log.info("Checking for installed CMake...")
p = subprocess.run(['which', 'cmake'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode == 0:
    log.info(f"CMake executable found at {p.stdout.decode()}")
else:
    log.warn("CMake executable not found! Installing ... ")
    cmd = ['brew', 'install', 'cmake'] if platform == 'darwin' else ['apt', 'install', 'cmake']
    log.info(f"Running: {' '.join(cmd)}...")
    p = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if p.returncode != 0:
        log.error(f"Failed to install cmake.\n"
                  f"Process STDOUT:\n{p.stdout.decode()}\n"
                  f"Process STDERR:\n{p.stderr.decode()}")
    else:
        log.info("Installation succeed!")

log.info("Installing required python modules...")
p = subprocess.run(['pip', 'install', '-r', 'requirements.txt'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error(f"Failed to install required python modules.\n"
              f"Process STDOUT:\n{p.stdout.decode()}\n"
              f"Process STDERR:\n{p.stderr.decode()}")
else:
    log.info("Installation succeed")

[ INFO ] Checking for installed CMake...
[ INFO ] CMake executable found at /usr/local/bin/cmake

[ INFO ] Installing required python modules...
[ INFO ] Installation succeed


Bellow you can find C library functions declaration. For more convenience listings with each function implmentation
will be added bellow right before using it from python.

c_library/library.h
```c
//  CFFI doesn't support directives in headers
//  So for compatibility reason all includes moved to .c files
//#include <stdio.h>
//#include <string.h>
//#include <unistd.h>

int func_ret_int(int val);
double func_ret_double(double val);
char *func_ret_str(char *val);
char func_many_args(int val1, double val2, char val3, short val4);


void arr_minus_one(void *data, unsigned int arr_size, unsigned int el_size, char *el_type);
int *gen_static_arr();
void gen_arr(int *arr, unsigned int size);
void qsort_wrap(int *arr, unsigned int size, unsigned int el_size);
int comp (const int *a, const int *b);
```
Here is brief description of each method and its purpose:
* Functions demonstrating working with basic types. Do nothing besides of receiving arguments of certain types,
printing basic information, which will show us that real C function was invoked, and returning some value.
```c
int func_ret_int(int val);
double func_ret_double(double val);
char *func_ret_str(char *val);
char func_many_args(int val1, double val2, char val3, short val4);
```
* Functions demonstrating how to manipulate with arrays:
    *  `void arr_minus_one(void *data, unsigned int arr_size, unsigned int el_size, char *el_type);` - receives an
    untyped array `void *data`, elements count in array `unsigned int arr_size`,
    element size in bytes `unsigned int el_size` and string representation of array's element data type `char *el_type`.
    The function iterates over passed array and subtract 1 from each element without memory copying.
    * `int *gen_static_arr();` - generates static array with 10 integer sequential elements starting from 0 and returns
    pointer to generated array
    * `void gen_arr(int *arr, unsigned int size);` - fill pre-allocated integer array `int *arr` of given size
    `unsigned int size` with integer values in range `[0, size - 1]`
    * `void qsort_wrap(int *arr, unsigned int size, unsigned int el_size);` - wraps standard quick sort algorithm from libc.
    Will be used for performance comparison between C, Python and mixed quick sort implementation
    * `int comp (const int *a, const int *b)` - C implementation of comparison function required by standard qsort
    function from libc


#### Building C library from sources.

In [4]:
import os
import shutil

TUTOR_ROOT = os.getcwd()
CMAKE_ROOT_FOLDER = os.path.join(TUTOR_ROOT, 'c_library',)
BUILD_FOLDER = os.path.join(CMAKE_ROOT_FOLDER, 'build')
if os.path.exists(BUILD_FOLDER):
    shutil.rmtree(BUILD_FOLDER)
os.mkdir(BUILD_FOLDER)

os.chdir(BUILD_FOLDER)
log.info("Running CMake...")
p = subprocess.run(['cmake', os.path.pardir], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error("CMake run failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info("CMake files generation succeed!\n"
             f"STDOUT:\n {p.stdout.decode()}\n")

log.info("Running library building with make...")
p = subprocess.run(['make'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if p.returncode != 0:
    log.error("Building failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info("Building suceed succeed!\n"
             f"STDOUT:\n {p.stdout.decode()}\n")

os.chdir(TUTOR_ROOT)

if platform in ("linux", "linux2"):
    C_LIB_NAME = "libbindings_demo.so"
elif platform == "darwin":
    C_LIB_NAME = "libbindings_demo.dylib"
else:
    raise OSError(f"Unsupported platform {platform}")
C_LIB_PATH = os.path.join(BUILD_FOLDER, C_LIB_NAME)
if not os.path.exists(C_LIB_PATH):
    raise FileNotFoundError(f"Compiled library not found at {C_LIB_PATH}")
else:
    log.info(f"Compiled library located at {C_LIB_PATH}")

[ INFO ] Running CMake...
[ INFO ] CMake files generation succeed!
STDOUT:
 -- The C compiler identification is AppleClang 12.0.0.12000031
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/mikhailtreskin/repos/py_bindings_tutorial/c_library/build


[ INFO ] Running library building with make...
[ INFO ] Building suceed succeed!
STDOUT:
 Scanning dependencies of target bindings_demo
[ 25%] Building C object CMakeFiles/bindings_demo.dir/basic_funcs.c.o
[ 50%] Building C object CMakeFiles/bindings_demo.dir/arrays.c.o
[ 75%] Building C object CMakeFiles/bindings_demo.dir/vars.c.o
[100%] Linking C shared library libbindings_demo.dylib
[100%] Built target bindings_demo


[ INFO ] Compiled library located at /Users/mikhailtreskin/

### Ctypes demo

[ctypes](https://docs.python.org/3/library/ctypes.html) is a foreign function library for Python. It provides C
compatible data types, and allows calling functions in DLLs or shared libraries. It can be used to wrap these libraries in pure Python.

What's stands for foreign function library? It's a library which implements foreign function interface (FFI) convention
between two certain languages - between C as a host language and Python as a guest language in case of ctypes.
Briefly, FFI is an interface that allows code written in one language to call code written in another language.
Ctypes rely on [libffi](https://github.com/libffi/libffi) library implementing FFI calling convention for C as a host language.

Ctypes allows to make calls to C functions and refere to C objects from Python in a relatively easy way. Let's go
through several examples to on how to interact with precompiled C library from Python using ctypes

First of all we have to load dynamic linked library in a following way (on Windows WINDLL call should be used instead CDLL)

In [5]:
import ctypes

lib = ctypes.CDLL(C_LIB_PATH)

And now we have an access to all of the functions mentioned above and declared in `c_library/library.h`.
Let's start from calling a simple functions consuming one or several "simple" arguments.

*Functions listed bellow are defined in `c_library/basic_funcs.c`*

Working with integers on example of pretty simple C function consuming integer argument performing logging print from C
code and then returns same integer value, which was provided as an input argument:
```c
int func_ret_int(int val) {
    printf("From C Library: func_ret_int: %d\n", val);
    return val;
}

double func_ret_double(double val) {
    printf("From C Library: func_ret_double: %f\n", val);
    return val;
}

```

To call functions from C library we have to describe it's signature in Python by specifying input arguments types
and return value type of desired C function. And then we can call it as an usual Python method.

In [6]:
# Jupyter notebook suppress default output from our C library so custom manager to capture stdout from C was added
from stdout_capture import capture_c_stdout

# Specifying return type of the func_ret_int C function
lib.func_ret_int.restype = ctypes.c_int
# Specifying input arguments type of the func_ret_int C function
lib.func_ret_int.argtypes = [ctypes.c_int, ]

# Same for double data type
lib.func_ret_double.restype = ctypes.c_double
lib.func_ret_double.argtypes = [ctypes.c_double]
with capture_c_stdout():
    print('From Python Script: func_ret_int: ', lib.func_ret_int(101))
    print('From Python Script: func_ret_double: ', lib.func_ret_double(12.123456789))

From C Library: func_ret_int: 101
From Python Script: func_ret_int:  101
From C Library: func_ret_double: 12.123457
From Python Script: func_ret_double:  12.123456789


The same for char functions like:
```c
char * func_ret_str(char *val) {
    printf("From C Library: func_ret_str: %s\n", val);
    return val;
}
```

On Python level the only important aspects is that we have to encode input strings to bytearray and then also decode
return value

In [7]:
lib.func_ret_str.restype = ctypes.c_char_p
lib.func_ret_str.argtypes = [ctypes.c_char_p, ]
with capture_c_stdout():
    print('From Python Script: func_ret_str: ', lib.func_ret_str('Hello!'.encode('utf-8')).decode("utf-8"))

From C Library: func_ret_str: Hello!
From Python Script: func_ret_str:  Hello!


Also nothing outstanding for functions with several input arguments with different data types, e.g.:

```c
char func_many_args(int val1, double val2, char val3, short val4) {
    printf("From C Library: func_many_args: int - %d, double - %f, char - %c, short - %d\n", val1, val2, val3, val4);
    return val3;
}
```
To call the function all input argument types have to be defined in proper order:

In [8]:
lib.func_many_args.restype = ctypes.c_char
lib.func_many_args.argtypes = [ctypes.c_int, ctypes.c_double, ctypes.c_char, ctypes.c_short]
with capture_c_stdout():
    print('From Python Script: func_many_args: ',
      lib.func_many_args(15, 18.1617, 'X'.encode('utf-8'), 32000).decode("utf-8"))

From C Library: func_many_args: int - 15, double - 18.161700, char - X, short - 32000
From Python Script: func_many_args:  X


Ctypes also allows to access and modify variables from C library. In `c_library/vars.h` we have defined following variables
```c
int a = 5;
double b = 5.12345;
char c = 'X';
```

To access variables from loaded C library we need ro use `in_dll()` class method of corresponding data type of variable
we going to access. `in_dll()` requires two input arguments - the 1st is an loaded library handler, and the 2nd is string
with variable name. E.g.:

In [9]:
a = ctypes.c_int.in_dll(lib, "a")
print('From Python Script: a: ', a.value)
b = ctypes.c_double.in_dll(lib, "b")
print('From Python Script: b: ', b.value)
c = ctypes.c_char.in_dll(lib, "c")
print('From Python Script: c: ', c.value.decode("utf-8"))


From Python Script: a:  5
From Python Script: b:  5.12345
From Python Script: c:  X


Variables exported form a library can be modified in a following way

In [10]:
print(f"'a' initial value: {a.value}")
a.value = 22
a = ctypes.c_int.in_dll(lib, "a")
print(f"'a' modified value: {a.value}")

'a' initial value: 5
'a' modified value: 22


OK, it were a simple examples which barely will be used in real life and were provided as introduction and way
to understand how ctypes works. But what about real life examples?

Bellow we will have a look on how to have a deal with C arrays basing on manipulation with numpy arrays.

### Updating existing array

*c_library/arrays.c*
```c
void arr_minus_one(int *data, unsigned int arr_size) {
    printf("From C Library:\n");
    for (int i = 0; i < arr_size; ++i) {
        data[i] = data[i] - 1;
        printf("%d ", data[i]);
    }
}
```
**Getting pointer from numpy array**
# TODO: INITIAL ARRAY NOT MODIFIED.
ctypes.data_as(ctypes.c_void_p) copies memory?

In [11]:
import numpy as np

arr = np.asarray([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.int32)
data_p = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
arr_size = arr.size
el_size = arr.itemsize
dtype = str(arr.dtype).encode('utf-8')

lib.arr_minus_one.restype = None
lib.arr_minus_one.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_uint]
print(f"Python numpy array:\n {arr}")
with capture_c_stdout():
    lib.arr_minus_one(data_p, arr_size, el_size, dtype)
print(f"Updated python numpy array:\n {arr}")

Python numpy array:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
From C Library:
0 1 2 3 4 5 6 7 8 Updated python numpy array:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]


**Getting pointer from python list**

In [12]:
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9]
data_p = (ctypes.c_int * len(arr))(*arr)
arr_size = 9
dtype = "int32".encode("utf-8")
print(f"Python list:\n {arr}")
with capture_c_stdout():
    lib.arr_minus_one(data_p, arr_size)
print(f"Updated python list:\n {arr}")

Python list:
 [1, 2, 3, 4, 5, 6, 7, 8, 9]
From C Library:
0 1 2 3 4 5 6 7 8 Updated python list:
 [1, 2, 3, 4, 5, 6, 7, 8, 9]


** Creating numpy array from pointer **

```c
int *gen_static_arr() {
    static int arr[10];
    for (int i = 0; i < 10; ++i) {
        arr[i] = i;
    }
    return arr;
}

void gen_arr(int *arr, unsigned int size) {
    for (int i = 0; i < size; ++i) {
        arr[i] = i;
    }
}
```

In [13]:
lib.gen_static_arr.restype = ctypes.POINTER(ctypes.c_int)
lib.gen_static_arr.argtypes = None
data_p = lib.gen_static_arr()

# Create from ctypes array
ctypes_arr = (ctypes.c_int * 10).from_address(ctypes.addressof(data_p.contents))
flat_arr = np.ctypeslib.as_array(ctypes_arr, shape=(2, 5))  # Shape ignored
print(f"Flat numpy array:\n {flat_arr}")
# Create from pointer directly
shaped_array = np.ctypeslib.as_array(data_p, shape=(5, 2))
print(f"Shaped numpy array:\n {shaped_array}")


Flat numpy array:
 [0 1 2 3 4 5 6 7 8 9]
Shaped numpy array:
 [[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]


```c
void gen_arr(int *arr, unsigned int size) {
    for (int i = 0; i < size; ++i) {
        arr[i] = i;
    }
}
```
**Create array from ctypes pointer**

In [14]:
lib.gen_arr.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_uint]
lib.gen_arr.restype = None

# Allocating memory
i_arr = (ctypes.c_int * 15)()
i_p = ctypes.cast(i_arr, ctypes.POINTER(ctypes.c_int))
lib.gen_arr(i_p, 15)
# Build numpy array
shaped_array = np.ctypeslib.as_array(i_p, shape=(3, 5))
print(f"Shaped numpy array:\n {shaped_array}")



Shaped numpy array:
 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


**Write directly to numpy array's memory**

In [15]:
arr = np.ndarray(shape=(5, 5), dtype=np.int32)
arr_p = arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
lib.gen_arr(arr_p, 25)
print(f"Shaped numpy array:\n {arr}")

Shaped numpy array:
 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]


**Find library and callbacks**

In [16]:
from ctypes.util import find_library
# libc and find_library
libc = ctypes.CDLL(find_library('c'))

a = (ctypes.c_int * 10)(5, 1, 7, 33, 99, 43, 12, 5, 1, 0)
libc.qsort.restype = None

# callbacks functions
func_prototype = ctypes.CFUNCTYPE(ctypes.c_int, ctypes.POINTER(ctypes.c_int), ctypes.POINTER(ctypes.c_int))


def py_cmp_func(a, b):
    return a[0] - b[0]


cmp_func = func_prototype(py_cmp_func)

libc.qsort(a, len(a), ctypes.sizeof(ctypes.c_int), cmp_func)
print([v for v in a])

[0, 1, 1, 5, 5, 7, 12, 33, 43, 99]


**Performance comparison**
quick sort in C, Python and mixture (C with Python callback)

In [17]:
from timeit import default_timer
import random

def py_qsort(nums):
    if len(nums) <= 1:
        return nums
    else:
        q = random.choice(nums)
    l_nums = [n for n in nums if n < q]

    e_nums = [q] * nums.count(q)
    b_nums = [n for n in nums if n > q]
    return py_qsort(l_nums) + e_nums + py_qsort(b_nums)

rand_arr = np.random.randint(low=0, high=500000, size=1000000, dtype=np.int32)

rand_list = list(rand_arr)
t0 = default_timer()
sorted_arr = py_qsort(rand_list)
t1 = default_timer()
print(f"Python qsort time: {t1 - t0}")
print(f"Sorted list slice {sorted_arr[0:100]}")

rand_data_p = rand_arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
t0 = default_timer()
libc.qsort(rand_data_p, rand_arr.size, ctypes.sizeof(ctypes.c_int), cmp_func)
t1 = default_timer()
print(f"libc qsort with callback py time: {t1 - t0}")
print(f"Sorted array slice {rand_arr[0:100]}")

rand_arr = np.random.randint(low=0, high=500000, size=1000000, dtype=np.int32)
rand_data_p = rand_arr.ctypes.data_as(ctypes.POINTER(ctypes.c_int))
lib.qsort_wrap.restype = None
t0 = default_timer()
lib.qsort_wrap(rand_data_p, rand_arr.size, ctypes.sizeof(ctypes.c_int))
t1 = default_timer()
print(f"libc qsort wrap without py callback time: {t1 - t0}")
print(f"Sorted array slice {rand_arr[0:100]}")

Python qsort time: 5.020982058000001
Sorted list slice [0, 1, 1, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 10, 10, 11, 11, 11, 11, 13, 13, 15, 16, 16, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 19, 20, 21, 21, 21, 22, 22, 23, 24, 24, 24, 25, 25, 25, 25, 25, 26, 27, 27, 28, 30, 31, 31, 31, 31, 31, 33, 33, 33, 33, 33, 33, 33, 34, 34, 34, 35, 35, 36, 37, 37, 37, 38, 39, 41, 41, 41, 43, 45, 45, 47, 47, 47, 48, 48, 49]
libc qsort with callback py time: 10.490331202000007
Sorted array slice [ 0  1  1  2  2  2  2  3  3  4  4  4  4  4  5  5  6  6  7  7  8  8  9 10
 10 11 11 11 11 13 13 15 16 16 17 17 17 17 17 17 18 18 18 18 19 20 21 21
 21 22 22 23 24 24 24 25 25 25 25 25 26 27 27 28 30 31 31 31 31 31 33 33
 33 33 33 33 33 34 34 34 35 35 36 37 37 37 38 39 41 41 41 43 45 45 47 47
 47 48 48 49]
libc qsort wrap without py callback time: 0.13749935799999946
Sorted array slice [ 0  2  2  3  3  4  4  4  4  4  5  5  5  6  7  8  9  9 10 10 10 11 11 11
 11 12 13 13 14 15 16 17 17 17 18 1

### CFFI demo

**Constants**

In [24]:
lib_path = "./cmake-build-debug/libbindings_demo.dylib"
include = os.path.join("./c_library", "library.h")

#### In-Line ABI Mode
Opens precompiled lib

In [56]:
import cffi
ffi = cffi.FFI()
with open(include) as f:
    ffi.cdef(f.read())

Library usage

In [57]:
t0 = default_timer()
lib = ffi.dlopen(lib_path)
t1 = default_timer()
log.info(f"In-line ABI import time {t1 - t0}")
with capture_c_stdout():
    res = lib.func_ret_str("Some str".encode("utf-8"))
    print(f"From Python Script: {ffi.string(res).decode('utf-8')}")

[ INFO ] In-line ABI import time 0.0005997599996589997


From C Library: func_ret_str: Some str
From Python Script: Some str


#### Out-of-Line ABI Mode
Opens precompiled lib with creating python module with binary info which speed-ups import time

In [60]:
# Need to reload cffi module and ffi object to allow to redefine header file and sources
# (Such trick required only for this jupyter notebook)
if "cffi" in globals():
    globals().pop("cffi")
if "ffi" in globals():
    globals().pop("ffi")
import cffi
ffi = cffi.FFI()
with open(include) as f:
    ffi.cdef(f.read())

ffi.set_source("cffi_libs.out_of_line_abi._bindings_demo", None)
ffi.compile()
target_module = "./cffi_libs/out_of_line_abi/_bindings_demo.py"
if not os.path.exists(target_module):
    raise FileNotFoundError(f"{target_module} not found! Possibly CFFI module compilation failed!")
log.info("Content of resulting folder")
print("./cffi_libs/out_of_line_abi/:")
for entry in os.listdir("./cffi_libs/out_of_line_abi/"):
    print(f"    {entry}")

[ INFO ] Content of resulting folder


./cffi_libs/out_of_line_abi/:
    __pycache__
    _bindings_demo.py


Created module usage

In [37]:
from cffi_libs.out_of_line_abi._bindings_demo import ffi as out_of_line_abi_ffi
t0 = default_timer()
lib = out_of_line_abi_ffi.dlopen(lib_path)
t1 = default_timer()
log.info(f"Out-of-line ABI import time {t1 - t0}")
with capture_c_stdout():
    res = lib.func_ret_str("Some str".encode("utf-8"))
    print(f"From Python Script: {ffi.string(res).decode('utf-8')}")

Out-of-line ABI import time 0.00011174399992341932
From C Library: func_ret_str: Some str
From Python Script: Some str


#### Out-of-Line API Mode
Compiles importable python library linked against our shared library.

In [63]:
if "cffi" in globals():
    globals().pop("cffi")
if "ffi" in globals():
    globals().pop("ffi")
import cffi
ffi = cffi.FFI()
with open(include) as f:
    ffi.cdef(f.read())
module_name = "_bindings_demo"
ffi.set_source(module_name,  # name of compiled library
               # Source code to be compiled along with CFFI-generated wrapper code
               f'#include "{include}"',
               # Libraries to link compiled CFFI module with (no extension and 'lib' prefix)
               # Passed directly to compiler as '-l' option
               libraries=["bindings_demo"],
               library_dirs=[os.path.dirname(lib_path)],
               )
target_dir = './cffi_libs/out_of_line_api'
ffi.compile(tmpdir=target_dir)

log.info("Content of resulting folder")
print(target_dir)
target_module_found = False
library_name = None
for entry in os.listdir(target_dir):
    print(f"    {entry}")
    if entry.startswith(module_name) and entry.endswith(".so"):  # both Linux and MacOS creates .so file
        library_name = entry
        target_module_found = True

if not target_module_found:
    raise FileNotFoundError(f"{target_module} library not found! Possibly CFFI module compilation failed!")

[ INFO ] Content of resulting folder


./cffi_libs/out_of_line_api
    _bindings_demo.cpython-38-darwin.so
    _bindings_demo.c
    _bindings_demo.o


Note that compiled python module dynamically linked against initial 'bindings_demo' library

In [67]:
if platform == "darwin":
    command = ['otool', '-L', os.path.join(target_dir, library_name)]
elif platform.startswith("linux"):
    command = ['ldd', os.path.join(target_dir, library_name)]
p = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

if p.returncode != 0:
    log.error("otool execution failed!\n"
              f"STDOUT:\n {p.stdout.decode()}\n"
              f"STDERR:\n {p.stderr.decode()}\n")
else:
    log.info(f"{library_name} library dependencies:\n"
             f"{p.stdout.decode()}\n")

[ INFO ] _bindings_demo.cpython-38-darwin.so library dependencies:
./cffi_libs/out_of_line_api/_bindings_demo.cpython-38-darwin.so:
	@rpath/libbindings_demo.dylib (compatibility version 0.0.0, current version 0.0.0)
	/usr/lib/libSystem.B.dylib (compatibility version 1.0.0, current version 1281.100.1)




Compiled module usage example

In [53]:
from cffi_libs.out_of_line_api._bindings_demo import ffi as out_of_line_api_ffi
from cffi_libs.out_of_line_api._bindings_demo import lib as out_of_line_api_lib

# dlopen not required
with capture_c_stdout():
    res = out_of_line_api_lib.func_ret_str("Some str".encode("utf-8"))
    print(f"From Python Script: {out_of_line_api_ffi.string(res).decode('utf-8')}")

From C Library: func_ret_str: Some str
From Python Script: Some str


#### Some extra usage examples

Another possible way of using Out-of-Line API mode which allows to avoid dynamic dependencies
is to specify all C sources right in `ffi.set_source()` method and do not add dependency libraries.
``` python
ffi.set_source("_my_module,  # name of compiled library
               # Source code to be compiled along with CFFI-generated wrapper code
               '''
                #include "library.h"

                #include <stdio.h>
                #include <string.h>
                #include <unistd.h>

                int func_ret_int(int val) {
                    printf("From C Library: func_ret_int: %d\n", val);
                    return val;
                }
                ''',
               # Libraries to link compiled CFFI module with (no extension and 'lib' prefix)
               # Passed directly to compiler as '-l' option
               libraries=[],
               )
```

Finally, you can (but don’t have to) use CFFI’s Distutils or Setuptools integration when writing a setup.py.

For Distutils (only in out-of-line API mode):
``` python
# setup.py (requires CFFI to be installed first)
from distutils.core import setup

import foo_build   # possibly with sys.path tricks to find it

setup(
    ...,
    ext_modules=[foo_build.ffibuilder.distutils_extension()],
)
```

For Setuptools (out-of-line, but works in ABI or API mode; recommended):
``` python
# setup.py (with automatic dependency tracking)
from setuptools import setup

setup(
    ...,
    setup_requires=["cffi>=1.0.0"],
    cffi_modules=["package/foo_build.py:ffibuilder"],
    install_requires=["cffi>=1.0.0"],
)
```
Note again that the foo_build.py example contains the following lines, which mean that the ffi builder is not
actually compiled when package.foo_build is merely imported—it will be compiled independently by the Setuptools logic,
using compilation parameters provided by Setuptools:

``` python
if __name__ == "__main__":    # not when running with setuptools
    ffibuilder.compile(verbose=True)
```

Using `...` in `ffi.cdef()` allows to skip some part of declarations in case of explicit declarations specification
(as a multiline string) and 'ask' C compiler to fill the gaps. Such way of declaration can be convenient in some
cases e.g. for `struct{}` or `union{}` declaration.
More details in the [documentation](https://cffi.readthedocs.io/en/latest/cdef.html#letting-the-c-compiler-fill-the-gaps).